# Linear dimensionality reduction with 
# Prinicipal Component Analysis on the ESDC
## by Max Planck Institute for Biogeochemistry
## M. Mahecha & F. Gans 

In [1]:
using Pkg
Pkg.add(PackageSpec(url="https://github.com/esa-esdl/ESDL.jl"))
Pkg.add(PackageSpec(url="https://github.com/esa-esdl/ESDLPlots.jl"))

    Updating git-repo `https://github.com/esa-esdl/ESDL.jl`
    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
    Updating git-repo `https://github.com/esa-esdl/ESDLPlots.jl`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`


In [2]:
using ESDL
using ESDLPlots

WebIO._IJuliaInit()

In this study we investigate the redundancy the different variables in each pixel. Therefore we calculate a linear dimensionality reduction (PCA) and check how many dimensions are needed to explain 90% of the variance of a cube that contained originally 6 variables.  First we check out the variables from the cube and add some processors, because we want to do a global study

## Access ESDC

In [3]:
c = esdc() #Cube()

YAXArray with the following dimensions
lon                 Axis with 1440 Elements from -179.875 to 179.875
lat                 Axis with 720 Elements from 89.875 to -89.875
time                Axis with 1840 Elements from 1979-01-05T00:00:00 to 2018-12-31T00:00:00
Variable            Axis with 69 elements: leaf_area_index sensible_heat .. snow_sublimation Rg 
units: W m-2
Total size: 490.37 GB


## Define variables for anaylsis

In [4]:
vars = ["gross_primary_productivity","latent_energy","par"
    ,"terrestrial_ecosystem_respiration","precipitation","max_air_temperature","net_ecosystem_exchange"];
cdata = subsetcube(c,variable=vars,region="Africa", time = 2001:2016);

## Gap-filling, needed to perform PCA

In [5]:
using Distributed
addprocs(5)
@everywhere using YAXArrays, Zarr, ESDL
cube_filled     = gapFillMSC(cdata);
#And we calculate the anomalies
cubeanom        = removeMSC(cube_filled)

Progress: 100%|█████████████████████████████████████████| Time: 0:07:44
Progress: 100%|█████████████████████████████████████████| Time: 0:00:06


YAXArray with the following dimensions
time                Axis with 736 Elements from 2001-01-05T00:00:00 to 2016-12-30T00:00:00
lon                 Axis with 272 Elements from -16.875 to 50.875
lat                 Axis with 320 Elements from 39.875 to -39.875
Variable            Axis with 7 elements: gross_primary_productivity latent_energy par terrestrial_ecosystem_respiration precipitation max_air_temperature_2m net_ecosystem_exchange 
Total size: 1.67 GB


In [7]:
#GC.gc()
#savecube(cube_filled, "data/filled_cube.zarr")
#savecube(cubeanom, "data/anomalies.zarr")


Progress: 100%|█████████████████████████████████████████| Time: 0:00:04


YAXArray with the following dimensions
time                Axis with 736 Elements from 2001-01-05T00:00:00 to 2016-12-30T00:00:00
lon                 Axis with 272 Elements from -16.875 to 50.875
lat                 Axis with 320 Elements from 39.875 to -39.875
Variable            Axis with 7 elements: gross_primary_productivity latent_energy par terrestrial_ecosystem_respiration precipitation max_air_temperature_2m net_ecosystem_exchange 
Total size: 1.67 GB


## Perform PCA

In [8]:
@everywhere using MultivariateStats, Statistics
@everywhere function sufficient_dimensions(xin::AbstractArray, expl_var::Float64 = 0.95)
    any(ismissing,xin) && return NaN
    npoint, nvar = size(xin)
    means = mean(xin,dims=1)
    stds  = std(xin,dims=1)
    xin   = broadcast((y,m,s)->s>0.0 ? (y-m)/s : one(y),xin,means,stds)
    pca = fit(PCA, xin', pratio = 0.999, method = :svd)
    return findfirst(cumsum(principalvars(pca)) / tprincipalvar(pca) .> expl_var)
end 

In [9]:
#works until here

#First we do the analysis on the original cube:
qualitypca=mapslices(sufficient_dimensions,cube_filled,0.90,dims = ("Time","Variable"));

┌ Warning: There are still cache misses
└ @ YAXArrays.DAT /home/anja/.julia/packages/YAXArrays/8cfQH/src/DAT/DAT.jl:978
Worker 5 terminated.
Unhandled Task ERROR: EOFError: read end of file
Stacktrace:
 [1] (::Base.var"#wait_locked#645")(s::Sockets.TCPSocket, buf::IOBuffer, nb::Int64)
   @ Base ./stream.jl:892
 [2] unsafe_read(s::Sockets.TCPSocket, p::Ptr{UInt8}, nb::UInt64)
   @ Base ./stream.jl:900
 [3] unsafe_read
   @ ./io.jl:724 [inlined]
 [4] unsafe_read(s::Sockets.TCPSocket, p::Base.RefValue{NTuple{4, Int64}}, n::Int64)
   @ Base ./io.jl:723
 [5] read!
   @ ./io.jl:725 [inlined]
 [6] deserialize_hdr_raw
   @ ~/Programme/julia-1.7.1/share/julia/stdlib/v1.7/Distributed/src/messages.jl:167 [inlined]
 [7] message_handler_loop(r_stream::Sockets.TCPSocket, w_stream::Sockets.TCPSocket, incoming::Bool)
   @ Distributed ~/Programme/julia-1.7.1/share/julia/stdlib/v1.7/Distributed/src/process_messages.jl:165
 [8] process_tcp_streams(r_stream::Sockets.TCPSocket, w_stream::Sockets.TCPSocket,

      From worker 4:	IOError: stream is closed or unusable
      From worker 4:	Stacktrace:
      From worker 4:	  [1] check_open
      From worker 4:	    @ ./stream.jl:386 [inlined]
      From worker 4:	  [2] uv_write_async(s::Sockets.TCPSocket, p::Ptr{UInt8}, n::UInt64)
      From worker 4:	    @ Base ./stream.jl:1018
      From worker 4:	  [3] uv_write(s::Sockets.TCPSocket, p::Ptr{UInt8}, n::UInt64)
      From worker 4:	    @ Base ./stream.jl:981
      From worker 4:	  [4] uv_write
      From worker 4:	    @ ./stream.jl:977 [inlined]
      From worker 4:	  [5] flush(s::Sockets.TCPSocket)
      From worker 4:	    @ Base ./stream.jl:1073
      From worker 4:	  [6] send_msg_(w::Distributed.Worker, header::Distributed.MsgHeader, msg::Distributed.RemoteDoMsg, now::Bool)
      From worker 4:	    @ Distributed ~/Programme/julia-1.7.1/share/julia/stdlib/v1.7/Distributed/src/messages.jl:187
      From worker 4:	  [7] send_msg
      From worker 4:	    @ ~/Programme/julia-1.7.1/share/julia/std

LoadError: ProcessExitedException(2)

      From worker 6:	  [1] check_open


# Result
## Complexity of the multivariate time series including the seasonal cycle
## How many variables are needed to explain 90% of the variance in the data?

In [ ]:
plotMAP(qualitypca,dmin=2,dmax=6)

And on the anomalies only:

In [ ]:
qualitypcaanom=mapslices(sufficient_dimensions,cubeanom,0.90, dims=("Time","Variable"));

## Complexity of the multivariate time series without the seasonal cycle

In [ ]:
plotMAP(qualitypcaanom,dmin=2,dmax=6)

In [ ]:
rmprocs(workers())